![Fluid-generated seismicity patterns.]<

# Animating the interplay of fluid circulation and seismicity in a subduction zone

*Date: 5th June 2020*

### As a foreword

This notebook allows you to animate a simulation of fluid diffusing in a fault zone, activating earthquake sources, modeled as valves. 

In our model, a valve represents inhomogeneities of permeability in the fault zone. They partly block the diffusion of fluid, but break open when submitted to high fluid pressures, producing a small earthquake. When the fluid flux around the valve comes back to base levels, the valve closes again, and the cycle goes on. The fundationnal ideas of the model are published in [this article](http://www.ipgp.jussieu.fr/~nshapiro/PUBLICATIONS/Shapiro_etal_GRL2018.pdf).

This allows us to model the interaction of elementary seismic sources uniquely through fluid interactions. From this simple physics, complex patterns of seismicity emerge. It reproduces the periodicity of activity observed in certain subduction zones and migrations of the seismic activity along the fault zone. 

This is work in progress! It is part of my PhD, and a publication is being written on it, but please do not hesitate to reach out for questions and comments, on the scientific or animation part. In any case, I hope you have fun playing with it, and can draw inspirations from whatever excites you in there.

Please don't hesitate to use this notebook and produced animations, to share it, and modify it for your own purpose, but be sure to mention where it comes from!

### A few words on the data we use here...

As this is solely simulation, all data that is used is purely synthetic, the product of a simulation of pore pressure diffusion in the system described above. For storage purposes, we drastically downsampled the data to the time steps that are used in each frames. The code works with this structure, but should be slightly adapted if a denser time series is used.

###  About me:

I am Gaspard Farge, PhD student in seismology and geological fluid dynamics at Institut de Physique du Globe de Paris. You can check out my research and other projects either on my [ResearchGate page](https://www.researchgate.net/profile/Gaspard_Farge), or on my [website](https://gfarge.github.io/) (rarely updated...). Feel free to contact me via email : farge@ipgp.fr.

---

## /// Import and paths

In [1]:
# General libraries 
# =================
import numpy as np
import pickle
import copy
import time
import sys

# Graphics libraries
# ==================
import matplotlib as mpl
%matplotlib notebook
import matplotlib.pyplot as plt
from matplotlib.ticker import (AutoMinorLocator, FixedLocator, FixedFormatter)
from matplotlib.patches import Rectangle
from matplotlib.collections import PatchCollection


# Animation libraries
# -------------------
import matplotlib.animation as animation
from matplotlib.animation import ArtistAnimation
plt.rcParams['animation.ffmpeg_path'] ='/usr/local/bin/ffmpeg'  # your path to ffmpeg binary
# --> Used for display within notebook
import base64
import io
from matplotlib import rc
rc('animation', html='html5')
from IPython.display import HTML # used to display animation in notebook

# My modules
# ==========
#sys.path.append('/Users/farge/work/py_modules/PPvalves/')
#import PPvalves.simulation as ppv
#import PPvalves.catalog as cat
#import PPvalves.utility as util

# Paths
# =====
in_dir = './data/'
anim_out_dir = './out/'

## /// Load data

In [2]:
# Paths
# =====
PARAM_file = in_dir + 'PARAM.pkl'
VALVES_file = in_dir + 'VALVES.pkl'
mass_file = in_dir + 'mass.npy'
v_states_file = in_dir + 'v_states.npy'
catalog_file = in_dir + 'catalog.npy'

In [3]:
# Load a few physical parameters of the simulation
# ================================================
PARAM = pickle.load(open(PARAM_file, 'rb'))
print('Physical parameters of the system are stored in the PARAM dictionnary.')
print('In there, you can find the following entries:\n\n{:}'.format(PARAM.keys()))

# Define a few variables that will be useful
# ==========================================
X = np.linspace(0, PARAM['Nx']*PARAM['h'], PARAM['Nx']+1)  # space array

Physical parameters of the system are stored in the PARAM dictionnary.
In there, you can find the following entries:

dict_keys(['h', 'dt', 'Nx', 'Nt', 'Z0', 'X_scale', 'T_scale', 'Z_scale'])


In [4]:
# Load valve properties
# =====================
VALVES = pickle.load(open(VALVES_file, 'rb'))
print('Valve properties are defined in the VALVES dictionnary.')
print('In there, you can find the following entries:\n\n{:}'.format(VALVES.keys()))

Valve properties are defined in the VALVES dictionnary.
In there, you can find the following entries:

dict_keys(['idx', 'width'])


In [5]:
# Load mass distribution in time/space
# ====================================
mass = np.load(mass_file)
print('The mass array contains the mass of fluid at each point of the domain, in time.')
print('First dimension corresponds to time, second dimension to space.')
print('\nIts shape: {:}'.format(np.shape(mass)))

The mass array contains the mass of fluid at each point of the domain, in time.
First dimension corresponds to time, second dimension to space.

Its shape: (1521, 501)


In [6]:
# Load all events in simulation (we will only use a few)
# ======================================================
catalog = np.load(catalog_file)
t_events = catalog[:, 0]  # event detection time
x_events = catalog[:, 1]  # location of event

print('The whole simulation contains {:} events.'.format(len(t_events)))
print('Their detection time are stored in t_event, their location are stored in x_event.')

The whole simulation contains 12678 events.
Their detection time are stored in t_event, their location are stored in x_event.


In [7]:
# Load valve states
# =================
v_states = np.load(v_states_file)
print('The opening/closing state of valves during the simulation is described by v_states')
print('\nv_states shape : N_times * N_valves: {:}'.format(np.shape(v_states)))
print('\nFirst dimension corresponds to time, last dimension to each valve.')
print('Valve state is encoded as follows : 1 is open, 0 is closed')

The opening/closing state of valves during the simulation is described by v_states

v_states shape : N_times * N_valves: (1521, 29)

First dimension corresponds to time, last dimension to each valve.
Valve state is encoded as follows : 1 is open, 0 is closed


## /// Make a valve class

In [8]:
# A class for valves graphical representation
# ===========================================
class valve(object):
    """The valve object"""
    
    def __init__(self, idx, idx_width, state):
        """Constructor function.
        Parameters
        ----------
        idx : int
            X index of valve start.
        idx_width : int
            X index of valve end.
        state : bool
            True if valve is open, False is valve is closed.
     
        """
        self.idx = [idx, idx + idx_width]
        self.open = state
        self.breaking = False
        self.set_color()
        
    
    def set_color(self):
        """Computes the alpha value for the valve patch (0 = fully transparent)"""
        if self.open:
            if self.breaking:
                self.color = 0.99
            else:
                self.color = 0.15
        else:
            self.color = 0.5
                
            
def build_valves(VALVES, v_state_0, PARAM):
    """
    Building a list of valve objects for all our valves.
    
    Parameters
    ----------
    VALVES : dict.
        Dictionnary of valve properties. In data.
    v_state_0 : 1d array.
        1d array describing valve initial state, dimension = N_valves.
    PARAM : dict
        Physical parameters dictionnary.
    
    Returns
    -------
    Valves : list of valve objects
    """
    Valves = []
    for iv in range(len(VALVES['idx'])):
        va = valve(VALVES['idx'][iv], \
                  int(VALVES['width'][iv]/PARAM['h']), \
                  bool(v_state_0[iv]))
                   
        Valves.append(va)
                   
    return Valves

## /// Make an event class

In [9]:
# A class for events graphical representation 
# ===========================================
class event(object):
    """Event dots class"""
    
    def __init__(self, t, x, valve_w):
        """
        Constructor.
        
        Parameters
        ----------
        t : float
            Event time.
        x : float
            Event X.
        valve_w : float
            Width of valve that created event.
        """
        self.fade = 0  # Level of fading after activation of event
        self.t = t  # Event detection/activation time
        self.x = x + valve_w/2  # Location of the event (center of valve)
        self.fade_time  = 0.01  # Fade is the proportion of fading from 0 no fade to 1 finished fading.
        self.max_size  = 100  # Maximum size of event dot
        self.min_size  = 1.5  # Minimum size of event dot


    def update(self, time):
        """Update the fade in time of events. time is current time."""
        # If event is active: current time is after event happens but 
        # before it disappears of screen
        # -----------------------------------------------------------
        if (self.t <= time) & (self.t + self.fade_time >= time ):
            self.fade = 1 - (time - self.t)/self.fade_time  # evolve fade
            
            self.set_color()
            self.set_size()
        
        # Otherwise
        # ---------
        else:
            self.fade = 0
            
            self.set_size()
            self.set_color()
            
    
    def set_size(self):
        """Compute size of event dot, function of its fade"""
        self.size = self.min_size + (self.max_size - self.min_size)*self.fade**3
    
    def set_color(self):
        """Compute color of event, function of its fade"""
        self.color = 0.5 + self.fade*0.5

def build_events(t_events, x_events, width_v):
    """
    Build an array of events. For now width should be the same for all valves (events).
    
    Parameters
    ----------
    t_events : 1d array
        Times of events.
    x_events : 1d array.
        Location of events.
    width_v : float
        Valve width.
    
    Returns
    -------
    Events : list of event objects
    """
    Events = []
    for t, x in zip(t_events, x_events):
        ev = event(t, x, width_v)
        Events.append(ev)
    
    return Events

## /// A function to initialize background

In [10]:
# Useful functions for the animation
# ==================================

def scale_mass(mass, plot_params):
    """ Scales mass with logistic curve: see larger scale rather than peaks """
    mass_sc = plot_params['sc_mass'] / (1 + np.exp(-1 * 3e6 * mass))
    
    return mass_sc

def droplet(x0, y0, a, b):
    """
    Draws a droplet using Longchamps piriforms
    https://math.stackexchange.com/questions/51539/a-math-function-that-draws-water-droplet-shape
    
    Parameters
    ----------
    x0, y0 : float
        Reference coordinates for the droplet. 
    a, b : float
        Parameters for the aspect of the droplet.
    Returns
    -------
    x_drop, y_drop : 1d arrays
        Coordinates for the drop shape. Dimension = 50.
    """
    
    t = np.linspace(0, 2*np.pi, num=50)
    x_drop = a*(1-np.sin(t))*np.cos(t) + x0
    y_drop = y0 - b*(np.sin(t)-1)  # for our plot, y is upside down
    
    return x_drop, y_drop

In [11]:
def background(X, mass0, Valves, plot_params, PARAM, VALVES):
    """
    Function to initialize animation background.
    
    Parameters
    ----------
    X : 1d array
        Space array, dimension PARAM['Nx'] + 1
    mass0 : 1d array
        Mass of fluid at every point of the domain at t0, dimension PARAM['Nx'] + 1
    Valves : list of valve objects
        Valve objects created before initializing.
    plot_params : dict
        Parameters for plotting.
    PARAM : dict.
        Dictionnary of physical and numerical parameters.
    VALVES : dict.
        Dictionnary of valve parameters.
    
    Returns
    -------
    fig : Matplotlib figure object
        The current figure.
    axes : list of matplotlib axes.
        Events and domain axes objects.
    g_objs : list of graphical objects
        PolyCollection for the fluid, PatchCollection for the valves.
    """
    
    # Initialize figure
    # =================
    fig = plt.figure(figsize=(8,6))
    gs = fig.add_gridspec(5,1)
    plt.subplots_adjust(hspace=0.05)


    fig.set_facecolor(plot_params['c_bg'])
    
    fig.suptitle('-- Drawing patterns --\n Swarms of seismic events in a subduction channel generated\n by the interplay of fluid diffusion and fault strength',
                 c=plot_params['c_ax'], y=0.99)
    
    # Events subplot
    # -------------
    ax_ev = fig.add_subplot(gs[:3,:])
    
    # --> Set colors of background and axes
    ax_ev.patch.set_color(plot_params['c_bg'])
    ax_ev.spines['left'].set_color(plot_params['c_ax'])
    ax_ev.spines['bottom'].set_visible(False)
    
    # --> Set up y axis ticks and labels: first deal with units
    week = 7 * 24 * 3600  # week duration in seconds
    day = 24 * 3600  # days duration in seconds
        
    T_tot = PARAM['Nt']*PARAM['dt']  # total duration of data
    dt_week = week / PARAM['T_scale'] # week length in adimensionalized unit
    dt_day = day / PARAM['T_scale'] # day length in adimensionalized unit
        
    # -->> Major ticks and labels
    T_ticks_weeks = np.arange(0, T_tot, dt_week)  # tick locations
    T_ticklabels_weeks = [str(int(t*PARAM['T_scale']/week)) 
                          for t in T_ticks_weeks]  # tick labels
    
    ax_ev.yaxis.set_major_locator(FixedLocator(T_ticks_weeks))
    ax_ev.yaxis.set_major_formatter(FixedFormatter((T_ticklabels_weeks)))
    
    ax_ev.tick_params(length=7, which='major')
       
    # -->> Minor ticks, no labels
    T_ticks_days = np.arange(0, T_tot, dt_day)  # tick locations
    
    ax_ev.yaxis.set_minor_locator(FixedLocator(T_ticks_days))
    
    ax_ev.tick_params(length=3, which='minor')

    # -->> A few more tick parameters
    ax_ev.tick_params(bottom=False,labelbottom=False,
                      left=True,labelleft=True,
                      colors=plot_params['c_ax'], which='both')
    
    ax_ev.grid(lw=0.8, c=[0.15, 0.15, 0.15], zorder=0, which='both')

    # --> Axes labels and limits
    ax_ev.set_ylabel('Event detection\n time (weeks)',
                     c=plot_params['c_ax'], size=plot_params['label_fs'])
    ax_ev.set_ylim(plot_params['t_lim'])
    ax_ev.set_xlim(plot_params['x_lim'])

    # Domain subplot
    # --------------
    ax_dom = fig.add_subplot(gs[3:,:])
    
    # --> Set axes and ticks
    ax_dom.patch.set_color(plot_params['c_bg'])
    ax_dom.spines['bottom'].set_color(plot_params['c_ax'])
    ax_dom.spines['left'].set_color(plot_params['c_ax'])
    ax_dom.spines['right'].set_visible('False')
    ax_dom.spines['top'].set_visible('False')
    
    # --> x/y axes parameters
    ax_dom.tick_params(left=True, labelleft=True,
                       colors=plot_params['c_ax'])
    
    ax_dom.set_xlim(plot_params['x_lim'])
    ax_dom.set_ylim(plot_params['z_lim'])
    
    ax_dom.set_xlabel('<-- Downdip / Along subduction channel (km) / Updip -->', 
                      c=plot_params['c_ax'], size=plot_params['label_fs'])
    ax_dom.set_ylabel('Depth (km)', 
                      c=plot_params['c_ax'], size = plot_params['label_fs'])
    
    # --> Plot fluid mass
    mass_sc = scale_mass(mass0, plot_params)  # emphasizing longer scale variations
    mass_plus = plot_params['Z_mass'] + mass_sc  # above curve
    mass_minus = plot_params['Z_mass'] - mass_sc  # below curve
    pc_dom = ax_dom.fill_between(X, mass_minus, mass_plus, 
                                 fc=plot_params['c_mass'], zorder=1)

    # --> In/out flux arrows and droplets
    arrowprops = dict(arrowstyle="->", fc=plot_params['c_mass'], \
                      ec=plot_params['c_mass'])
    ax_dom.annotate("", xytext=(-0.09, 1.09), xy=(-0.01, 1.01), \
                    arrowprops=arrowprops)
    ax_dom.annotate("", xytext=(1.01,-.01), xy=(1.09, -0.09), \
                    arrowprops=arrowprops)
    
    x_drop1, y_drop1 = droplet(-0.06, 0.93, 0.005, 0.04)
    x_drop2, y_drop2 = droplet(1.05, 0, 0.005, 0.04)

    ax_dom.plot(x_drop1, y_drop1, '-', lw=1, c=plot_params['c_mass'])
    ax_dom.plot(x_drop2, y_drop2, '-', lw=1, c=plot_params['c_mass'])
    
    # --> Plot valves
    valve_boxes = [Rectangle( (xv, plot_params['z_lim'][0]), wv,
                              plot_params['z_lim'][1]-plot_params['z_lim'][0])
                   for xv, wv in zip(X[VALVES['idx']], VALVES['width'])]
    
    valves_ptc = PatchCollection(valve_boxes, alpha=1, edgecolor=None,
                                 axes=ax_dom,zorder=0)
    valves_ptc.set_facecolor([plot_params['c_v'](v.color) 
                              for v in Valves])
    
    ax_dom.add_collection(valves_ptc)
    
    # --> Change ticks to dimensionnalized values
    X_ticks = np.array([0, 0.2, 0.4, 0.6, 0.8, 1]) # here pretty straight forward
    
    ax_dom.set_xticks(X_ticks)
    ax_dom.set_xticklabels([str(int(x*PARAM['X_scale']/1000)) 
                            for x in X_ticks])
    
    Z_ticks = np.array([40, 41, 42, 43])*1000 / PARAM['Z_scale'] - PARAM['Z0']
    ax_dom.set_yticks(Z_ticks)
    ax_dom.set_yticklabels(['40', '41', '42', '43'])
    
    # Pack up objects for handling later on
    # -------------------------------------
    axes  = [ax_ev, ax_dom]
    gobjs = [pc_dom, valves_ptc]
    
    return fig, axes, gobjs

## /// Functions to update context

### *(a)* A function to update events

In [12]:
def update_events(time, Events, t_lim):
    """
    A function to get active events and update their fade.
    
    Parameters
    ----------
    time : float
        Current time.
    Events : list of event objects
        The events to update.
    t_lim : float
        Current top limit of event graph.
    
    Returns
    -------
    active_ev : list
        List of currently active events, that is, events that happened and
        are still within plotting bounds.
    """
    # Get active events
    # -----------------
    active_ev = [ev for ev in Events 
                 if ((ev.t < time) & (ev.t > t_lim))]
    
    # Update fade of active events
    # ----------------------------
    for ev in active_ev:
        ev.update(time)
    
    return active_ev

### *(b)* A function to update valves

In [13]:
def update_valves(it, previous, v_states, Valves):
    """
    Update valve states and dP at t=time
    
    Parameters
    ----------
    it : float
        Current time index.
    previous : float
        Previous time
    v_activity : 3d array
        Valve activity from the data. Describes valve activity.
    Valves : list of valve objects
        Current valves.

    
    Returns
    -------
    Valves : list of valve objects
        Updated valve objects.
    """
    # --> Unpack
    itp  = it - 1  # index of previous time
    
    # --> Update
    for iv, va in enumerate(Valves):
        # --> If it was breaking previously, it's not now
        if va.breaking:
            va.breaking = False
        
        # -->> Update valve state and check if breaking
        va.open = bool(v_states[it, iv])
        # -->>> Breaking = previous = 0 (closed), current = 1 (open)
        if (v_states[it-1, iv] - v_states[it, iv]) == -1:
            va.breaking = True
        
        # -->> Update color
        va.set_color()
    
    return Valves

## /// A function to update the plots 

In [14]:
def update_anim(time, Valves, active_ev, X, mass, fig, axes, valves_ptc, plot_params):
    """
    Update the plots for the current time
    
    Parameters
    ----------
    time : float
        Current time.
    Valves : list of valve objects
        Valve objects created before initializing. Output of update_valves.
    active_ev : list of event objects
        Current active objects. Output of update_events.
    X : 1d array
        Space array, dimension PARAM['Nx'] + 1
    mass : 1d array
        Mass of fluid at every point of the domain at time t, dimension PARAM['Nx'] + 1
    fig : Matplotlib figure object
        Current figure.
    axes : list of matplotlib axes.
        Events and domain axes objects.
    valves_ptc
    
    Returns
    -------
    fig : Matplotlib figure object
        The current figure.
    axes : list of matplotlib axes.
        Events and domain axes objects.
    valves_ptc, pc_ev, pc_dom : graphical objects
        PatchCollection for the valves, PathCollection for the events, PolyCollection for the fluid
    """
    
    # --> Unpack
    ax_ev, ax_dom = axes
    
    # Update events plot
    # ------------------
    # --> Update plot limits
    ax_ev.set_ylim(time + plot_params['t_fact'] * plot_params['dt_lim'],
                   time - (1 - plot_params['t_fact']) * plot_params['dt_lim'])
    
    # --> Plot current events
    ev_x = [ev.x for ev in active_ev]
    ev_t = [ev.t for ev in active_ev]
    ev_s = [ev.size for ev in active_ev]
    ev_c = [plot_params['c_ev'](ev.color) for ev in active_ev]
    pc_ev = ax_ev.scatter(ev_x, ev_t, s=ev_s, c=ev_c, 
                          edgecolors='face', zorder=10)
    
    # Update domain plot
    # ------------------
    # --> Update barriers
    v_fc = [plot_params['c_v'](va.color) for va in Valves]
    valves_ptc.set_facecolor(v_fc)
    
    # --> Update flux
    mass_sc = scale_mass(mass, plot_params)
    mass_plus = plot_params['Z_mass'] + mass_sc
    mass_minus = plot_params['Z_mass'] - mass_sc
    pc_dom = ax_dom.fill_between(X, mass_minus, mass_plus, 
                                 fc=plot_params['c_mass'], zorder=1)

    return fig, axes, valves_ptc, pc_ev, pc_dom

## /// Set plot parameters

In [15]:
# Choosing plot parameters 
# ========================
plot_params = {}

# Mass related properties
# -----------------------
plot_params['Z_mass']  = np.linspace(1, 0, num=len(X))
plot_params['sc_mass'] = 0.1

# Plot colors
# -----------
plot_params['c_bg'] = 'k'
plot_params['c_ax'] = 'w'
plot_params['c_v']  = plt.cm.gray
plot_params['c_ev'] = plt.cm.afmhot
plot_params['c_mass'] = 'aquamarine'

# Limits
# -------
plot_params['v_h'] =  0.2 # valve patch height
plot_params['x_lim']  = (-0.1, 1.1)
plot_params['z_lim']  = (plot_params['Z_mass'][0]*1.15, plot_params['Z_mass'][-1]-0.15)
plot_params['dt_lim'] = 0.012
plot_params['t_fact'] = 0.05  # proportion of axes where events appear
plot_params['t_lim']  = (0 + plot_params['dt_lim']*plot_params['t_fact'],
                         0 - plot_params['dt_lim']*(1 - plot_params['t_fact']))

# Fontsizes
# ---------
plot_params['label_fs'] = 13

## /// Set animation parameters

In [16]:
# Animation parameters 
# ====================
animname = 'fluid_swarms.switch.test'
dpi = 200

# Choose either a period to cover or a target_time
# ------------------------------------------------
t0 = 0.43  # time at which to start
t1 = 0.534  # time at which to switch to slow spee
tn = 0.5812 # end time of animation

period_fast = t1 - t0
period_slow = tn - t1

fps = 24
frame_dt_slow = .00005  # slow.short
frame_dt_fast = .00018   # fast.short

N_frames_fast = int(period_fast / frame_dt_fast)
N_frames_slow = int(period_slow / frame_dt_slow)
N_frames = N_frames_slow + N_frames_fast
target_duration = N_frames / fps
print('Animation {:} frames long, for a total duration of {:.2f} s'.format(N_frames, target_duration))
print('--> Fast action: {:.2f}'.format(N_frames_fast/fps))
print('--> Slow action: {:.2f}'.format(N_frames_slow/fps))

# Writer set up
# -------------
anim_filename = anim_out_dir + animname +'.mp4'
metadata = dict(title='Movie Test', artist='Matplotlib',comment='Movie support!')                                                                                   
writer = animation.FFMpegWriter(fps=fps, bitrate=1e9)

Animation 1521 frames long, for a total duration of 63.38 s
--> Fast action: 24.04
--> Slow action: 39.33


## /// Animate

In [17]:
# Initialize 
# ==========
# Get minimal value of M in animation range, set it to 0
# ------------------------------------------------------

# Build objects
# -------------
Valves = build_valves(VALVES, v_states[0, :], PARAM)
Events = build_events(t_events, x_events, VALVES['width'][0])

# Plot background
# ---------------
plt.close()
mpl.use('Agg')

fig, axes, gobjs = background(X, mass[0, :], Valves, plot_params, PARAM, VALVES)
axes[1].collections.remove(gobjs[0])
valves_ptc = gobjs[1]

frame_dt = frame_dt_fast  # start with fast action
ti = t0
switch = True

indices_out = []
# Animation loop
# ==============
with writer.saving(fig, anim_filename, dpi):
    print('Writing frames...')
    for ii in range(1, N_frames):
        # The current time is...
        # ----------------------
        time = ti + ii * frame_dt
        previous = ti + (ii-1) * frame_dt
        
        #indices_out.append(int(time/PARAM['dt_']))  # gather indices to produce data easier to manipulate
        
        if (time > t1) and switch:  # transition to slow action
            frame_dt = frame_dt_slow
            ti = time - (ii * frame_dt_slow)
            switch = False
        
        # Update the events and valves
        # ----------------------------
        active_ev = update_events(time, Events, time - plot_params['dt_lim'])
        Valves    = update_valves(ii, previous, v_states, Valves)
    
        # Update the plot
        # ---------------
        fig, axes, valves_ptc, ev_coll, M_coll = update_anim(time, Valves, active_ev, X,
                                                             mass[ii, :],
                                                             fig, axes, valves_ptc, plot_params)
    
        # Grab the frame
        # --------------
        writer.grab_frame(facecolor='k')
    
        # Delete the past events
        # ----------------------
        axes[0].collections.remove(ev_coll)
        axes[1].collections.remove(M_coll)

print('Done!\n')
plt.close()

Writing frames...
Done!



In [ ]:
# Show 
# ----
video = io.open(anim_filename, 'r+b').read()
encoded = base64.b64encode(video)
HTML(data='''<video width="800" height="800" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii')))